In [ ]:
from typing import Optional, Dict, Any
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import optuna
import numpy as np
import pandas as pd

In [ ]:
class FeatureEngineering:
    
    def _ship_related_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Splits 'Cabin' into 'Deck' and 'Cabin_part'.
        
        Args:
            data (pd.DataFrame): The input DataFrame containing the 'Cabin' column.

        Returns:
            pd.DataFrame: The updated DataFrame with 'Deck' and 'Cabin_part' columns.
        """
        # Split 'Cabin' column into 'Deck' and 'Cabin_part' directly
        data[['Deck', 'Cabin_part']] = data['Cabin'].str.split("/", expand=True).iloc[:, [0, 2]]

        return data

    def _passenger_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Creates passenger-related features including group size and family details.
        
        Args:
            data (pd.DataFrame): The input DataFrame containing passenger information.

        Returns:
            pd.DataFrame: The updated DataFrame with additional passenger features.
        """
        # Calculate the number of passengers per cabin and per group in one go using transform
        data['NUMBER_OF_PASSENGERS_PER_CABIN'] = data.groupby('Cabin')['PassengerId'].transform('count')
        data['NUMBER_OF_PASSENGERS_PER_GROUP'] = data['PassengerId'].str.split("_").str[0].map(
            data['PassengerId'].str.split("_").str[0].value_counts()
        )
        data['Avg_Age_Per_Group'] = data.groupby('NUMBER_OF_PASSENGERS_PER_GROUP')['Age'].transform('mean')

        # Create 'wasAlonePerGroup' and 'wasAlonePerCabin' columns based on conditions
        data['wasAlonePerGroup'] = (data['NUMBER_OF_PASSENGERS_PER_GROUP'] == 1).astype(int)
        data['wasAlonePerCabin'] = (data['NUMBER_OF_PASSENGERS_PER_CABIN'] == 1).astype(int)

        # Extract last name directly and calculate family-related columns
        data['LAST_NAME'] = data['Name'].str.split().str[1]
        data['FAMILY'] = data['LAST_NAME'].map(data['LAST_NAME'].value_counts())
        data['TRAVELLED_WITH_FAMILY'] = (data['FAMILY'] > 1).astype(int)

        # Drop temporary columns if they are no longer needed
        data.drop(columns=['FAMILY'], inplace=True)

        return data

    def _service_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Calculates service-related spending features.
        
        Args:
            data (pd.DataFrame): The input DataFrame containing service spending information.

        Returns:
            pd.DataFrame: The updated DataFrame with service-related features.
        """
        service_columns = ['RoomService', 'Spa', 'FoodCourt', 'ShoppingMall', 'VRDeck']

        mean_spending_pass = data.groupby('PassengerId')[service_columns].mean()
        median_spending_pass = data.groupby('PassengerId')[service_columns].median()
        total_spending_pass = data.groupby('PassengerId')[service_columns].sum()

        mean_spending_fam = data.groupby('LAST_NAME')[service_columns].mean()
        median_spending_fam = data.groupby('LAST_NAME')[service_columns].median()
        total_spending_fam = data.groupby('LAST_NAME')[service_columns].sum()


        for column in service_columns:
            data[f'Mean_Spending_On_{column}_Pass'] = data['PassengerId'].map(mean_spending_pass[column])
            data[f'Median_Spending_On_{column}_Pass'] = data['PassengerId'].map(median_spending_pass[column])
            data[f'Total_Spending_On_{column}_Pass'] = data['PassengerId'].map(total_spending_pass[column])

            data[f'Mean_Spending_On_{column}_Fam'] = data['LAST_NAME'].map(mean_spending_fam[column])
            data[f'Median_Spending_On_{column}_Fam'] = data['LAST_NAME'].map(median_spending_fam[column])
            data[f'Total_Spending_On_{column}_Fam'] = data['LAST_NAME'].map(total_spending_fam[column])

        data['isMinor'] = (data['Age'] < 18).astype(int)
        data['Age_Cat'] = data['Age'].apply(lambda x: 'Child' if x <= 12 else 
                                              'Teen' if x < 18 else 
                                              'Adult' if x < 64 else 'Senior')
        data['isCyroSleep'] = data['CryoSleep'].apply(lambda x: 1 if x == 'True' else 0)
        data['isVIP'] = data['VIP'].apply(lambda x: 1 if x == 'True' else 0)
        data['Average_Family_Size'] = data.groupby('LAST_NAME')['PassengerId'].transform('count')

        return data
    
    def _handling_missing_values(self, data: pd.DataFrame) -> pd.DataFrame:
        """Handles missing values in the dataset.
        
        Args:
            data (pd.DataFrame): The input DataFrame to handle missing values.

        Raises:
            NotImplementedError: This method is not yet implemented.
        """
        raise NotImplementedError("This method is not yet implemented!")
    
    def _destination_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Creates destination related features.
        
        Args:
            data (pd.DataFrame): The input DataFrame containing destination related features.

        Returns:
            pd.DataFrame: The updated DataFrame with destination related features. """

        data['Destination_Count_By_HomePlanet'] = data.groupby('Destination')['HomePlanet'].transform('count')

        return data 

    def _feature_pipeline(self, data: pd.DataFrame) -> pd.DataFrame:
        """Runs the full feature engineering pipeline on data.
        
        Args:
            data (pd.DataFrame): The input DataFrame to process.

        Returns:
            pd.DataFrame: The processed DataFrame with all features engineered.
        """
        data_processed = self._ship_related_features(data)
        data_processed = self._passenger_features(data_processed)
        data_processed = self._service_features(data_processed)
        data_processed = self._destination_features(data_processed)

        return data_processed
    
    def _data_split(self, data: pd.DataFrame, features: list[str], target: str) -> tuple[pd.DataFrame, pd.Series]:
        """Splits the data into features and target.
        
        Args:
            data (pd.DataFrame): The input DataFrame containing features and target.
            features (list[str]): The list of feature column names.
            target (str): The name of the target column.

        Returns:
            tuple[pd.DataFrame, pd.Series]: A tuple containing the features DataFrame and target Series.
        """
        X_train = data[features]
        y_train = data[target]

        return X_train, y_train
    
    def _get_dummies(self, data: pd.DataFrame, dtype: type) -> pd.DataFrame:
        """Creates dummy variables for categorical features.
        
        Args:
            data (pd.DataFrame): The input DataFrame to create dummy variables from.
            dtype (type): The desired data type for the resulting dummy variables.

        Returns:
            pd.DataFrame: The DataFrame with dummy variables added.
        """
        dummied_data = pd.get_dummies(data, dtype=dtype)
        
        return dummied_data


In [ ]:
class Modelling:
    """
    A class to handle machine learning model initialization and hyperparameter tuning using Optuna.

    Attributes:
        model (Any): The initialized machine learning model.
        available_models (Dict[str, Any]): A dictionary mapping model names to their respective classes.
    """

    def __init__(self):
        """
        Initializes the Modelling class and defines available models for initialization.
        """
        self.model = None
        self.available_models = {
            'RandomForestClassifier': RandomForestClassifier, 
            'XGBClassifier': XGBClassifier,
            'LGBMClassifier': LGBMClassifier,
        }

    def initialize_model(self, model_name: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """
        Initializes and returns a machine learning model based on the model_name.

        Args:
            model_name (str): Name of the model to initialize.
            params (Optional[Dict[str, Any]]): Hyperparameters for the model. Defaults to None.

        Returns:
            Any: An instance of the selected model.

        Raises:
            ValueError: If the model_name is not in available_models.
        """
        if model_name not in self.available_models:
            print(f"The {model_name} is not yet available, you can select a following model: {list(self.available_models.keys())}")
            raise ValueError(f"{model_name} is not a valid model.")
        
        
        return self.available_models[model_name](**(params or {}))
    
    def predict_test_set(self, model: Any, X_test: pd.DataFrame) -> np.ndarray:

        """
        Predicts the target variable for the given test set using the provided model.

        Args:
            model (Any): The trained model used for making predictions.
            X_test (pd.DataFrame): The test set features for which predictions are to be made.

        Returns:
            np.ndarray: The predicted values for the test set.
        """
        predictions = model.predict(X_test)
        return predictions

    def fit_model(self, model: Any, X_train: pd.DataFrame, y_train: pd.Series) -> Any:
        """
        Fits the provided model to the training data.

        Args:
            model (Any): The model to be trained.
            X_train (pd.DataFrame): The training set features.
            y_train (pd.Series): The target variable corresponding to the training set features.

        Returns:
            Any: The fitted model.
        """
        return model.fit(X_train, y_train)

        
    def tune_model(self, model_name: str, X_train: Any, y_train: Any, n_trials: int) -> Dict[str, Any]:
        """
        Tunes the specified model's hyperparameters using Optuna.

        Args:
            model_name (str): Name of the model to tune.
            X_train (Any): Training features.
            y_train (Any): Training labels.
            n_trials (int): Number of trials for hyperparameter tuning.

        Returns:
            Dict[str, Any]: The best parameters found during tuning.

        Raises:
            ValueError: If the model_name is not in available_models.
        """

        optuna.logging.set_verbosity(optuna.logging.WARNING)

        if model_name not in self.available_models:
            print(f"The {model_name} is not yet available, you can select a following model: {list(self.available_models.keys())}")
            raise ValueError(f"{model_name} is not a valid model.")

        if model_name == 'RandomForestClassifier':
            return self.tune_random_forest(X_train, y_train, n_trials)

        elif model_name == 'XGBClassifier':
            return self.tune_xgb(X_train, y_train, n_trials)

        elif model_name == 'LGBMClassifier':
            return self.tune_lgbm(X_train, y_train, n_trials)

        else:
            raise ValueError(f"{model_name} is not a valid model.")

    def tune_random_forest(self, X_train: Any, y_train: Any, n_trials: int) -> Dict[str, Any]:
        """
        Tunes hyperparameters for the RandomForestClassifier.

        Args:
            X_train (Any): Training features.
            y_train (Any): Training labels.
            n_trials (int): Number of trials for hyperparameter tuning.

        Returns:
            Dict[str, Any]: The best parameters found during tuning.
        """
        def rfc_objective(trial):
            n_estimators = trial.suggest_int("n_estimators", 50, 500)
            max_depth = trial.suggest_int("max_depth", 3, 20)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
            bootstrap = trial.suggest_categorical("bootstrap", [True, False])
            criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
            max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 10, 100)

            rf = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                bootstrap=bootstrap,
                criterion = criterion,
                max_leaf_nodes = max_leaf_nodes,
                random_state=42,
            )

            score = cross_val_score(rf, X_train, y_train, cv=5, scoring="accuracy").mean()
            return score

        rfc_study = optuna.create_study(direction="maximize")
        rfc_study.optimize(rfc_objective, n_trials=n_trials)

        print("Best parameters for RandomForestClassifier:", rfc_study.best_params)
        print("Best score for RandomForestClassifier:", rfc_study.best_value)

        return rfc_study.best_params

    def tune_xgb(self, X_train: Any, y_train: Any, n_trials: int) -> Dict[str, Any]:
        """
        Tunes hyperparameters for the XGBClassifier.

        Args:
            X_train (Any): Training features.
            y_train (Any): Training labels.
            n_trials (int): Number of trials for hyperparameter tuning.

        Returns:
            Dict[str, Any]: The best parameters found during tuning.
        """
        def xgb_objective(trial):
            n_estimators = trial.suggest_int("n_estimators", 50, 500)
            max_depth = trial.suggest_int("max_depth", 3, 15)
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            subsample = trial.suggest_float("subsample", 0.5, 1.0)
            colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
            gamma = trial.suggest_float("gamma", 0, 5)
            reg_alpha = trial.suggest_float("reg_alpha", 0, 10)
            reg_lambda = trial.suggest_float("reg_lambda", 0, 10)
            scale_pos_weight = trial.suggest_float("scale_pos_weight", 0.5, 1.0)

            xgb = XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                subsample=subsample,
                colsample_bytree=colsample_bytree,
                gamma=gamma,
                reg_alpha=reg_alpha,
                reg_lambda=reg_lambda,
                scale_pos_weight=scale_pos_weight,
                random_state=42,
            )

            score = cross_val_score(xgb, X_train, y_train, cv=5, scoring="accuracy").mean()
            return score

        xgb_study = optuna.create_study(direction="maximize")
        xgb_study.optimize(xgb_objective, n_trials=n_trials)

        print("Best parameters for XGBClassifier:", xgb_study.best_params)
        print("Best score for XGBClassifier:", xgb_study.best_value)

        return xgb_study.best_params

    def tune_lgbm(self, X_train: Any, y_train: Any, n_trials: int) -> Dict[str, Any]:
        """
        Tunes hyperparameters for the LGBMClassifier.

        Args:
            X_train (Any): Training features.
            y_train (Any): Training labels.
            n_trials (int): Number of trials for hyperparameter tuning.

        Returns:
            Dict[str, Any]: The best parameters found during tuning.
        """
        def lgbm_objective(trial):
            n_estimators = trial.suggest_int("n_estimators", 50, 500)
            max_depth = trial.suggest_int("max_depth", -1, 15)  # -1 means no limit
            learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
            num_leaves = trial.suggest_int("num_leaves", 2, 256)
            min_child_samples = trial.suggest_int("min_child_samples", 5, 100)
            subsample = trial.suggest_float("subsample", 0.5, 1.0)
            colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
            reg_alpha = trial.suggest_float("reg_alpha", 0, 10)
            reg_lambda = trial.suggest_float("reg_lambda", 0, 10)

            lgbm = LGBMClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                num_leaves=num_leaves,
                min_child_samples=min_child_samples,
                subsample=subsample,
                colsample_bytree=colsample_bytree,
                reg_alpha=reg_alpha,
                reg_lambda=reg_lambda,
                random_state=42,
            )

            score = cross_val_score(lgbm, X_train, y_train, cv=5, scoring="accuracy").mean()
            return score

        lgbm_study = optuna.create_study(direction="maximize")
        lgbm_study.optimize(lgbm_objective, n_trials=n_trials)

        print("Best parameters for LGBMClassifier:", lgbm_study.best_params)
        print("Best score for LGBMClassifier:", lgbm_study.best_value)

        return lgbm_study.best_params


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.dropna(inplace=True)

In [ ]:
FEATURES = ['Deck', 'Cabin_part', 'NUMBER_OF_PASSENGERS_PER_CABIN', 'NUMBER_OF_PASSENGERS_PER_GROUP', 'wasAlonePerGroup', 'wasAlonePerCabin', 'TRAVELLED_WITH_FAMILY', 'isMinor', 'Age_Cat',
       'Mean_Spending_On_RoomService_Pass', 'Mean_Spending_On_Spa_Pass',
       'Mean_Spending_On_FoodCourt_Pass', 'Mean_Spending_On_ShoppingMall_Pass',
       'Mean_Spending_On_VRDeck_Pass' ,'Mean_Spending_On_RoomService_Fam', 'Mean_Spending_On_Spa_Fam',
       'Mean_Spending_On_FoodCourt_Fam', 'Mean_Spending_On_ShoppingMall_Fam',
       'Mean_Spending_On_VRDeck_Fam','isCyroSleep', 'HomePlanet', 'isVIP', 'Destination', 'Destination_Count_By_HomePlanet',]
TARGET = ['Transported']

In [ ]:
feature_engineering = FeatureEngineering()
modelling = Modelling()

In [ ]:
processed_train = feature_engineering._feature_pipeline(train)
processed_test = feature_engineering._feature_pipeline(test)
X_train, y_train = feature_engineering._data_split(processed_train, features = FEATURES, target = TARGET)

X_train = pd.get_dummies(X_train, dtype='int')
y_train = pd.get_dummies(y_train, dtype='int')
X_test = pd.get_dummies(processed_test[FEATURES], dtype='int')



In [ ]:
lgb_best_optuna_params = modelling.tune_model('LGBMClassifier', X_train=X_train, y_train=y_train, n_trials=300)


In [ ]:
LGB_MODEL = modelling.initialize_model(model_name='LGBMClassifier', params=lgb_best_optuna_params)
LGB_MODEL_fitted = modelling.fit_model(model = LGB_MODEL, X_train=X_train, y_train=y_train)

test['Transported'] = modelling.predict_test_set(model = LGB_MODEL_fitted, X_test=X_test)
test['Transported'] = test['Transported'].apply(lambda x: True if x == 1 else False)
submission = test[['PassengerId', 'Transported']]